In [ ]:
from bs4 import BeautifulSoup, SoupStrainer
import requests
import urllib.request
import copy
import pandas as pd
from newspaper import Article
import re
import nltk
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
import string
import pandas as pd
import re 
path_save_real_news='real/'
# словарь для отнесения к теме
data = pd.read_csv('vocab/vocab.csv')
stopwords_ru=list(data['0'])
# символы для удаления
patterns = "[!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
morph = MorphAnalyzer()

In [ ]:
# новостей про ИБ с кждого сайта 
it_news_count=200
# новостей не про ИБ с каждого сайта
simpl_news_count=200
# процент слов из словаря, который должна содержать статья
percent=0.05

In [ ]:
# лемматизация
def lemmatize(doc):
    global percent
    doc = re.sub(patterns, ' ', doc)
    tokens = []
    for token in doc.split():
        if token and token not in stopwords_ru:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            tokens.append(token)
    raznost=len(doc.split())-len(tokens)
    if  raznost >= round(percent*len(doc.split())):
        return 0
    else:
        return 112

In [1]:
# получить текст новостной статьи и проверяем его на принадлежность теме
# 0 - текст новости
# 1 - относится ли к ИБ (1-да, 0- нет)
# 2 - является фейком (1-да,0 - нет)
# 3 - ссылка на новость
def get_news(url, data_,flag,datc):
    news = Article(url, language='ru')
    news.download()
    news.parse()
    data_frame=pd.DataFrame()
    el=str(news.text)
    if flag==1:
        text = copy.deepcopy(el[el.find("\n",el.find("\n")+2)+2:].lower())
    else:
        text=copy.deepcopy(el)
    data_frame=pd.DataFrame()
    data_frame=pd.concat([data_frame,pd.DataFrame([text])])
    flag=data_frame[0].apply(lemmatize)
    if (datc==0 or datc==1) and flag[0]==0:
        data_=pd.concat([data_,pd.DataFrame({'News':[copy.deepcopy(el)], 'InfSec':[1],'FAKE?':[0],'Link':[copy.deepcopy(url)]})])
        return data_,0
    elif (datc==0 or datc==2) and flag[0]!=0:
        data_=pd.concat([data_,pd.DataFrame({'News':[copy.deepcopy(el)], 'InfSec':[0],'FAKE?':[0],'Link':[copy.deepcopy(url)]})])
        return data_,112
    return None,None

In [2]:
def flag_func(flag,it_news,simpl_news,datchik):
    #отображает какие новости добавлять: 0-все, 1-ИБ, 2-простые 
    #(нужно чтобы когда закончили добавлять один тип, продолжили искать другой)
    global it_news_count
    global simpl_news_count
    if flag==0:
        it_news=it_news+1
    else:
        simpl_news=simpl_news+1
    if datchik==0:
        if it_news>=it_news_count:
            datchik=2
        if simpl_news>=simpl_news_count:
            datchik=1
    return it_news,simpl_news,datchik

In [ ]:
# далее ддля каждого сайта из за различной разметки
# _page - вынает количество страниц с сайта
# _news - осуществляет изъятие текстов новостей

In [ ]:
# www.securitylab.ru

In [ ]:
# выясняем сколько страниц на сайие
def secure_lab_page():
    url="https://www.securitylab.ru/news/page1_1.php"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'}
    result = requests.get(url, headers=headers)
    # поиск по тегам
    soup = BeautifulSoup(result.text, 'html.parser', parse_only=SoupStrainer('div'))
    box = soup.find('div', class_='page-picker')
    st = str(box.find('input'))
    num_page=''
    for i in range(12,len(st)):
        if st[i]=='"':
            break
        else:
            num_page=num_page+st[i]
    num_page=int(num_page)
    return num_page

In [ ]:
# получаем новости
def secure_lab_news(num_page):
    global it_news_count
    global simpl_news_count
    it_news=0
    simpl_news=0
#     отображает какие новости добавлять: 0-все, 1-ИБ, 2-простые
    datchik=0
    dataframe=pd.DataFrame()
    for k in range(1,num_page+1):
        url="https://www.securitylab.ru/news/page1_"+str(k)+".php"
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'}
        result = requests.get(url, headers=headers)
        soup = BeautifulSoup(result.text, 'html.parser', parse_only=SoupStrainer('div'))
        box = soup.find_all('a', class_='article-card inline-card')
        link=[]
        for i in range (0, len(box)):
            try:
                stroka_url=str(box[i])
                urls=''
                for j in range (42,len(stroka_url)):
                    if stroka_url[j]!='"':
                        urls=urls+stroka_url[j]
                    else:
                        temp_dataframe,flag=get_news("https://www.securitylab.ru"+urls,dataframe,1,datchik)
                        if flag!=None:
                            dataframe=copy.deepcopy(temp_dataframe)
                        break
                if flag!=None:
                    it_news,simpl_news,datchik=flag_func(flag,it_news,simpl_news,datchik)
                    if it_news>=it_news_count and simpl_news>=simpl_news_count:
                        print('len dataframe=',len(dataframe),' it_news=',it_news,' simpl_news=',simpl_news,"  странициа сайта=",k)
                        return dataframe
            except Exception:
                e=1
        print('len dataframe=',len(dataframe),' it_news=',it_news,' simpl_news=',simpl_news,"  странициа сайта=",k)
    return dataframe

In [ ]:
# xakep.ru

In [ ]:
# выясняем сколько страниц на сайие
def xakep_page():
    url="https://xakep.ru/category/news/"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'}
    result = requests.get(url, headers=headers)
    # поиск по тегам
    soup = BeautifulSoup(result.text, 'html.parser', parse_only=SoupStrainer('div'))
    box = soup.find('span', class_='pages')
    st=str(box)
    num_page=''
    for i in range(34,len(st)):
        if st[i]=='<':
            break
        elif re.search(r'[0-9]', st[i]):
            num_page=num_page+st[i]
    num_page=int(num_page)
    return num_page

In [ ]:
# получаем новости
def xakep_news(num_page):
    global it_news_count
    global simpl_news_count
    it_news=0
    simpl_news=0
#     отображает какие новости добавлять: 0-все, 1-ИБ, 2-простые
    datchik=0
    dataframe=pd.DataFrame()
    for k in range(1,num_page+1):
        url="https://xakep.ru/category/news/page/"+str(k)+"/"
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'}
        result = requests.get(url, headers=headers)
        soup = BeautifulSoup(result.text, 'html.parser', parse_only=SoupStrainer('div',class_='bdaia-blocks-container'))
        box = soup.find_all('h3',class_='entry-title')
        link=[]
        for i in range (0, len(box)):
            try:
                stroka_url=str(box[i])
                urls=''
                for j in range (33,len(stroka_url)):
                    if stroka_url[j]!='"':
                        urls=urls+stroka_url[j]
                    else:
                        temp_dataframe,flag=get_news(urls,dataframe,2,datchik)
                        if flag!=None:
                            dataframe=copy.deepcopy(temp_dataframe)
                        break
                if flag!=None:
                    it_news,simpl_news,datchik=flag_func(flag,it_news,simpl_news,datchik)
                    if it_news>=it_news_count and simpl_news>=simpl_news_count:
                        print('len dataframe=',len(dataframe),' it_news=',it_news,' simpl_news=',simpl_news,"  странициа сайта=",k)
                        return dataframe
            except Exception:
                e=1
        print('len dataframe=',len(dataframe),' it_news=',it_news,' simpl_news=',simpl_news,"  странициа сайта=",k)
    return dataframe

In [ ]:
# www.anti-malware.ru

In [ ]:
# выясняем сколько страниц на сайие
def anti_malware_page():
    url="https://www.anti-malware.ru/news?page=0"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'}
    result = requests.get(url, headers=headers)
    # поиск по тегам
    soup = BeautifulSoup(result.text, 'html.parser', parse_only=SoupStrainer('li',class_='pager-last'))
    box = soup.find('a')
    st=str(box)
    num_page=''
    for i in range(20,len(st)):
        if st[i]=='"':
            break
        elif re.search(r'[0-9]', st[i]):
            num_page=num_page+st[i]
    num_page=int(num_page)
    return num_page

In [ ]:
# получаем новости
def anti_malware_news(num_page):
    global it_news_count
    global simpl_news_count
    it_news=0
    simpl_news=0
#     отображает какие новости добавлять: 0-все, 1-ИБ, 2-простые
    datchik=0
    dataframe=pd.DataFrame()
    for k in range(0,num_page):
        url="https://www.anti-malware.ru/news?page="+str(k)
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'}
        result = requests.get(url, headers=headers)
        soup = BeautifulSoup(result.text, 'html.parser', parse_only=SoupStrainer('div',class_='region region-content'))
        box = soup.find_all('h2')
        link=[]
        for i in range (0, len(box)):
            try:
                stroka_url=str(box[i])
                urls=''
                for j in range (13,len(stroka_url)):
                    if stroka_url[j]!='"':
                        urls=urls+stroka_url[j]
                    else:
                        temp_dataframe,flag=get_news('https://www.anti-malware.ru'+urls,dataframe,2,datchik)
                        if flag!=None:
                            dataframe=copy.deepcopy(temp_dataframe)
                        break
                if flag!=None:
                    it_news,simpl_news,datchik=flag_func(flag,it_news,simpl_news,datchik)
                    if it_news>=it_news_count and simpl_news>=simpl_news_count:
                        print('len dataframe=',len(dataframe),' it_news=',it_news,' simpl_news=',simpl_news,"  странициа сайта=",k)
                        return dataframe
            except Exception:
                e=1
        print('len dataframe=',len(dataframe),' it_news=',it_news,' simpl_news=',simpl_news,"  странициа сайта=",k)
    return dataframe

In [ ]:
# securitymedia.org

In [ ]:
# выясняем сколько страниц на сайие
def securitymedia_page():
    url="https://securitymedia.org/news/?PAGEN_2=1"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'}
    result = requests.get(url, headers=headers)
    # поиск по тегам
    soup = BeautifulSoup(result.text, 'html.parser', parse_only=SoupStrainer('div',class_='modern-page-navigation'))
    box = soup.find_all('a')
    st=str(box[len(box)-2])
    num_page=''
    for i in range(24,len(st)):
        if st[i]=='"':
            break
        else:
            num_page=num_page+st[i]
    num_page=int(num_page)
    return num_page

In [ ]:
# получаем новости
def securitymedia_news(num_page):
    global it_news_count
    global simpl_news_count
    it_news=0
    simpl_news=0
#     отображает какие новости добавлять: 0-все, 1-ИБ, 2-простые
    datchik=0
    dataframe=pd.DataFrame()
    for k in range(0,num_page):
        url="https://securitymedia.org/news/?PAGEN_2="+str(k)
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'}
        result = requests.get(url, headers=headers)
        soup = BeautifulSoup(result.text, 'html.parser', parse_only=SoupStrainer('div',class_='col-md-9'))
        box = soup.find_all('a',class_="text-dark")
        link=[]
        for i in range (0, len(box),2):
            try:
                stroka_url=str(box[i])
                urls=''
                
                for j in range (27,len(stroka_url)):
                    if stroka_url[j]!='"':
                        urls=urls+stroka_url[j]
                    else:
                        temp_dataframe,flag=get_news('https://securitymedia.org'+urls,dataframe,2,datchik)
                        if flag!=None:
                            dataframe=copy.deepcopy(temp_dataframe)
                        break
                if flag!=None:
                    it_news,simpl_news,datchik=flag_func(flag,it_news,simpl_news,datchik)
                    if it_news>=it_news_count and simpl_news>=simpl_news_count:
                        print('len dataframe=',len(dataframe),' it_news=',it_news,' simpl_news=',simpl_news,"  странициа сайта=",k)
                        return dataframe
            except Exception:
                e=1
        print('len dataframe=',len(dataframe),' it_news=',it_news,' simpl_news=',simpl_news,"  странициа сайта=",k)
    return dataframe

In [ ]:
result_data1frame=pd.DataFrame()
result_data2frame=pd.DataFrame()
result_data3frame=pd.DataFrame()
result_data4frame=pd.DataFrame()

In [ ]:
page=securitymedia_page()
site_pd=securitymedia_news(copy.deepcopy(page))
result_data1frame=pd.concat([site_pd,result_data1frame])

In [ ]:
page=anti_malware_page()
# тут страницы с нулевой
site_pd=anti_malware_news(copy.deepcopy(page+1))
result_data2frame=pd.concat([site_pd,result_data2frame])

In [ ]:
page=xakep_page()
site_pd=xakep_news(copy.deepcopy(page))
result_data3frame=pd.concat([site_pd,result_data3frame])

In [ ]:
page=secure_lab_page()
site_pd=secure_lab_news(copy.deepcopy(page))
result_data4frame=pd.concat([site_pd,result_data4frame])

In [ ]:
result_data1frame.to_csv(path_save_real_news+'dataset1_news.csv', encoding='utf-8', index=False)
result_data2frame.to_csv(path_save_real_news+'dataset2_news.csv', encoding='utf-8', index=False)
result_data3frame.to_csv(path_save_real_news+'dataset3_news.csv', encoding='utf-8', index=False)
result_data4frame.to_csv(path_save_real_news+'dataset4_news.csv', encoding='utf-8', index=False)

In [ ]:
result_dataframe=pd.DataFrame()
result_dataframe=pd.concat([result_dataframe,result_data1frame]).reset_index(drop=True)
result_dataframe=pd.concat([result_dataframe,result_data2frame]).reset_index(drop=True)
result_dataframe=pd.concat([result_dataframe,result_data3frame]).reset_index(drop=True)
result_dataframe=pd.concat([result_dataframe,result_data4frame]).reset_index(drop=True)

In [ ]:
# result_dataframe.to_csv('dataset_news.csv', encoding='utf-8', index=False)

In [ ]:
# len(result_dataframe)